# Import

In [1]:
import csv
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_mutual_info_score

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
tqdm.pandas()

# Read data

In [3]:
ag_news_df = pd.read_csv("../../../Preprocessing/data/AgNews/master.csv", index_col=0)

In [4]:
with open("../../../Preprocessing/data/AgNews/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [5]:
vector_sizes = [2, 3, 4, 6, 7, 8, 9, 10, 20, 40, 80, 160, 320, 640]

In [6]:
label = ag_news_df["class"].to_numpy()

In [7]:
model_nums = 2
vector_model_num = 0

In [8]:
def gmm_value(gmm, vectors, label):
    pred = gmm.predict(vectors)
    aic = gmm.aic(vectors)
    bic = gmm.bic(vectors)
    mi = adjusted_mutual_info_score(pred, label)
    logl = gmm.score(vectors, label)
    return {"pred": pred, "aic":aic, "bic":bic, "mi":mi, "logl":logl}

In [9]:
vectors_path = "../../../Clustering/data/AgNews/doc2vec/vector/"
models_path = "../../../Clustering/data/AgNews/doc2vec/GMM/model/"
pred_path = "../../../Clustering/data/AgNews/doc2vec/GMM/pred/"
stats_list = []
for model_num in range(model_nums):
    stats = []
    for vector_size in tqdm(vector_sizes):
        vector_path = f"{vectors_path}{vector_size}/normalized/{vector_model_num}.csv"
        vectors = np.loadtxt(vector_path, delimiter=",")
        
        gmm = pickle.load(open(f"{models_path}{vector_size}/{model_num}.sav", "rb"))
        value = gmm_value(gmm, vectors, label)
        pred, *stat = value.items()
        stats.append(dict(stat))
    stats_list.append(stats)

  0%|          | 0/14 [00:03<?, ?it/s]


AttributeError: 'KMeans' object has no attribute 'aic'

In [ ]:
stats_list = [pd.DataFrame(stats, index=vector_sizes) for stats in stats_list]

In [ ]:
stats_path = "../../data/AgNews/doc2vec/GMM/stats/"
for model_num in range(model_nums):
    os.makedirs(f"{stats_path}", exist_ok=True)
    stats_list[model_num].to_csv(f"{stats_path}{model_num}.csv")

In [ ]:
stats_list[0]